In [1]:
!pip install transformers
!pip install tokenizers
!pip install nltk
!pip install pandas
!pip install scipy
!pip install scikit-learn

## Resume

In [2]:
import pandas as pd
df = pd.read_csv('/content/emoticon12.csv')

df.head()

total_emojis =[]

emoji_dictionary = pd.DataFrame(columns=['emoji', 'meaning', 'sentiment'])

for i in range(len(df['icons'])):
  str_list_of_emojis = df['icons'][i]
  str_list_of_emojis = str_list_of_emojis[1:-1]
  #print(str_list_of_emojis)
  list_of_emojis = str_list_of_emojis.split(",")
  #print(list_of_emojis)
  list_of_emojis = [emoji.strip() for emoji in list_of_emojis]
  #print(list_of_emojis)
  list_of_emojis = [emoji[1:-1] for emoji in list_of_emojis]
  #print(list_of_emojis)
  for emoji in list_of_emojis:
    emoji_dictionary.loc[len(emoji_dictionary)] = [emoji, df['meaning'][i], df['Sentiment'][i]]

  total_emojis.extend(list_of_emojis)
print(total_emojis)

[':-]', '8-)', '8-)', '=]', '^^', '^ . ^', ':-)', '><', '>_<', ':)', ':-D', ':D', '=D', ":')", 'xd', 'x-D', 'xD', ':-))', ':))', ':-((', ':((', ':{', ':@', ':(', ":'(", ":'-D", ":'D", ':<', ':O', 'x3', ':3', ';]', ';D', 'xp', ':-p', ':p', 'd:', '=p', ':-/', ':/', '=/', ':-|', ':|', '8-)', '8)', '</3', '<3', 'o7', '-.-', 'qq', 'x_x', 'o-o', 'o_o', ' - _ - )', ' - __ - )', ' . _ . )']


In [3]:
emoji_dictionary

,emoji,meaning,sentiment
0,:-],Happy,1
1,8-),Happy,1
2,8-),Happy,1
3,=],Happy,1
4,^^,Happy,1
5,^ . ^,Happy,1
6,:-),Happy,1
7,><,Happy,1
8,>_<,Happy,1
9,:),Happy,1


In [4]:

emojis_with_parentheses = []
for emoji in total_emojis:
  if '(' in emoji or ')' in emoji:
    if emoji != ":)" and emoji != ":(" and emoji != ":((" and emoji != ":))":
      emojis_with_parentheses.append(emoji)
print(emojis_with_parentheses)



['8-)', '8-)', ':-)', ":')", ':-))', ':-((', ":'(", '8-)', '8)', ' - _ - )', ' - __ - )', ' . _ . )']


In [6]:
import numpy as np
import pandas as pd

with open("train_neg.txt") as f:
    neg_txt = [line for line in f]
    neg_df = pd.DataFrame(neg_txt, columns=['line']).drop_duplicates(keep='first')
    print(neg_df.head())
f.close()

                                                line
0  vinco tresorpack 6 ( difficulty 10 of 10 objec...
1  glad i dot have taks tomorrow ! ! #thankful #s...
2  1-3 vs celtics in the regular season = were fu...
3  <user> i could actually kill that girl i'm so ...
4  <user> <user> <user> i find that very hard to ...


In [7]:
with open("train_pos.txt") as f:
    pos_txt = [line for line in f]
    pos_df = pd.DataFrame(pos_txt, columns=['line']).drop_duplicates(keep='first')
    print(pos_df.head())
f.close

                                                line
0  <user> i dunno justin read my mention or not ....
1  because your logic is so dumb , i won't even c...
2  " <user> just put casper in a box ! " looved t...
3  <user> <user> thanks sir > > don't trip lil ma...
4  visiting my brother tmr is the bestest birthda...


<function TextIOWrapper.close()>

In [8]:
pos_df['label'] = 1
neg_df['label'] = 0

In [9]:
full_df = pd.concat([pos_df,neg_df],ignore_index=True)
full_df.tail()

,line,label
181316,can't wait to fake tan tonight ! hate being pa...,0
181317,<user> darling i lost my internet connection ....,0
181318,kanguru defender basic 4 gb usb 2.0 flash driv...,0
181319,rizan is sad now\n,0
181320,"no text back ? yea , he mad\n",0


In [10]:
import re
import nltk
from tqdm import tqdm
corpus = []
for i in tqdm(range(len(full_df))):
  #review = re.sub('[^a-zA-Z]', ' ', full_df['line'][i])
  #review = review.lower()
  #review = review.split()
  #review = ' '.join(review)
  review = full_df['line'][i]
  review = review.lower()
  corpus.append(review)

100%|██████████| 181321/181321 [00:02<00:00, 85833.65it/s]


In [11]:
def detect_dyck_language(s):
    stack = []
    for index, char in enumerate(s):
        if char == '(':
            stack.append((char, index))
        elif char == ')':
            if not stack:
                return (index, ')')
            stack.pop()

    if stack:
        _, index = stack.pop()
        return (index,'(')
    else:
        return None

In [12]:

def twitter_specific_analysis(line,replaced):
  replace = False
  if "( <user> live on <url>" in line:
    line = line.replace("( <user> live on <url>", "")

  if "( <user> <url>" in line:
    line = line.replace("( <user> <url>", "")

  if "( <url>" in line:
    line = line.replace("( <url>","")

  if "( rt <user>" in line:
    line = line.replace("( rt <user>","")

  if "( via <user>" in line:
    line = line.replace("( via <user>","")

  if ") ) )" in line:
    line = line.replace(") ) )","excited")

  if "< ^ , ^ )" in line:
    line = line.replace("< ^ , ^ )","happy")


  for emoji in emojis_with_parentheses:
    if emoji in line:
      row = emoji_dictionary.loc[emoji_dictionary['emoji'] == emoji]
      replacement = row['meaning'].values[0]
      #print(replacement)
      line = line.replace(emoji, replacement)

      #print(line)

  return line


In [ ]:
"""
def get_char_surrounded(index,line):
    if index-1>=0:
      char_prev = line[index-1]
    else:
      char_prev = ' '
    if index+1<len(line):
      char_next = line[index+1]
    else:
      char_next = ' '
    char_concat = char_prev + char + char_next
"""

In [13]:
import time
count_easy = 0
start = time.time()
new_corpus_with_smileys = []
replaced = 0
for i, line in enumerate(corpus):
  line = twitter_specific_analysis(line,replaced)

  res = detect_dyck_language(line)
  if res is not None:
    index,char = res

    if char == '(':
      substring = ':('
      newline = line[:index] + substring + line[index:]
      new_corpus_with_smileys.append(newline)
      #print(newline)
    elif char ==')':
      substring = ':)'
      newline = line[:index] + substring + line[index:]
      new_corpus_with_smileys.append(newline)
      #print(newline)
    else:
      new_corpus_with_smileys.append(line)

  else:
    new_corpus_with_smileys.append(line)

print(len(new_corpus_with_smileys))


181321


In [15]:

full_df['line'] = new_corpus_with_smileys

full_df.to_csv("data_frame_with_semilables4.csv", index=False)

## First step classify every thing with :) and :(

## Step 2 : Calculate for each line, if it contains any of the emojis. If so give them sentiment score which is 1 or 0 for postive/negative sentiment. And create a dataframe that contains

In [20]:
corpus = full_df['line']
labels = full_df['label']

fake_labels = []
emoji_scores = []
new_corpus = []
for index, line in enumerate(corpus):
    pred_fake_lables = 0
    if ':)' in line:
        pred_fake_lables = 1
    elif ':(' in line:
        pred_fake_lables = -1


    words = set(line.split(" "))
    emoji_score =0
    present_emoji = words & set(total_emojis)
    if len(present_emoji) > 0:
        for emoji in present_emoji:
            row = emoji_dictionary.loc[emoji_dictionary['emoji'] == emoji]
            replacement = row['meaning'].values[0]
            line = line.replace(emoji, replacement)
            if emoji_dictionary[emoji_dictionary['emoji'] == emoji]['sentiment'].values[0] == 1:
              emoji_score += 1

            elif emoji_dictionary[emoji_dictionary['emoji'] == emoji]['sentiment'].values[0] == 0:
              emoji_score -= 1
    new_corpus.append(line)

    if emoji_score > 0:
      emoji_score = 1
    elif emoji_score < 0:
      emoji_score = -1
    else:
      emoji_score = 0
    fake_labels.append(pred_fake_lables)
    emoji_scores.append(emoji_score)

print(len(fake_labels))
print(len(emoji_scores))

full_df['fake_lables'] = fake_labels
full_df['emoji_scores'] = emoji_scores
full_df['line'] = new_corpus
filtered_df = full_df[full_df['emoji_scores'] > 0]

filtered_df.head()


181321
181321


,line,label,fake_lables,emoji_scores
6,<user> #1dnextalbumtitle : feel for you / roll...,1,0,1
26,my summer days : 1 . Very happy work from 10:3...,1,1,1
34,<user> thank you again love & & i def will get...,1,1,1
52,<user> i haven't seen myself though and they d...,1,0,1
57,<user> watching now hehe Very happy )\n,1,1,1


In [21]:
filtered_df2 = full_df[full_df['emoji_scores'] < 0]

filtered_df2.head()

,line,label,fake_lables,emoji_scores
498,<user> until i am in melbourne and available f...,1,-1,-1
770,tweetiung from the harmonys Very Sad lol\n,1,-1,-1
890,<user> i know ! Annoyed hey did you know that ...,1,0,-1
1515,<user> haha yesss ! ! ! Very Sad the exclamati...,1,-1,-1
1581,Very Sad ~ <user> thanx for the follow hit * l...,1,-1,-1


In [22]:
filtered_df.tail()

,line,label,fake_lables,emoji_scores
180792,<user> chillax bi help me to trend this i'm be...,0,0,1
180810,"( ) rt <user> yessir < - , - Very happy rt <us...",0,1,1
180985,<user> hello bffl love and idk i went on after...,0,0,1
181248,"<user> if they will do it , say to bella ! Lau...",0,0,1
181309,<user> its not problem follow <user> .. its pr...,0,0,1


## Test accuracy

In [26]:
filtered_df_final = full_df[(full_df['emoji_scores'] != 0) & (full_df['fake_lables'] != 0)]
# Count based on the provided conditions
count_label_1_and_both_1 = len(filtered_df_final[(filtered_df_final['label'] == 1) & (filtered_df_final['emoji_scores'] == 1) & (filtered_df_final['fake_lables'] == 1)])
count_label_1_and_either_1 = len(filtered_df_final[(filtered_df_final['label'] == 1) & ((filtered_df_final['emoji_scores'] == 1) | (filtered_df_final['fake_lables'] == 1))])
count_label_1_and_neither_1 = len(filtered_df_final[(filtered_df_final['label'] == 1) & (filtered_df_final['emoji_scores'] != 1) & (filtered_df_final['fake_lables'] != 1)])

count_label_0_and_both_minus_1 = len(filtered_df_final[(filtered_df_final['label'] == 0) & (filtered_df_final['emoji_scores'] == -1) & (filtered_df_final['fake_lables'] == -1)])
count_label_0_and_either_minus_1 = len(filtered_df_final[(filtered_df_final['label'] == 0) & ((filtered_df_final['emoji_scores'] == -1) | (filtered_df_final['fake_lables'] == -1))])
count_label_0_and_neither_minus_1 = len(filtered_df_final[(filtered_df_final['label'] == 0) & (filtered_df_final['emoji_scores'] != -1) & (filtered_df_final['fake_lables'] != -1)])

print("Count of rows with label 1 and both emoji_score and fake_labels are 1:", count_label_1_and_both_1)
print("Count of rows with label 1 and either emoji_score or fake_labels is 1:", count_label_1_and_either_1)
print("Count of rows with label 1 and neither emoji_score nor fake_labels is 1:", count_label_1_and_neither_1)

print("Count of rows with label 0 and both emoji_score and fake_labels are -1:", count_label_0_and_both_minus_1)
print("Count of rows with label 0 and either emoji_score or fake_labels is -1:", count_label_0_and_either_minus_1)
print("Count of rows with label 0 and neither emoji_score nor fake_labels is -1:", count_label_0_and_neither_minus_1)

Count of rows with label 1 and both emoji_score and fake_labels are 1: 2104
Count of rows with label 1 and either emoji_score or fake_labels is 1: 2119
Count of rows with label 1 and neither emoji_score nor fake_labels is 1: 290
Count of rows with label 0 and both emoji_score and fake_labels are -1: 27305
Count of rows with label 0 and either emoji_score or fake_labels is -1: 27321
Count of rows with label 0 and neither emoji_score nor fake_labels is -1: 234


In [27]:
len(filtered_df_final)

29964

In [28]:
full_df.to_csv("data_frame_with_semilables5.csv", index=False)

In [31]:
wrong_pos = 1-count_label_1_and_neither_1/count_label_1_and_either_1
wrong_neg = 1-count_label_0_and_neither_minus_1/count_label_0_and_either_minus_1
print(wrong_pos)
print(wrong_neg)

0.8631429919773478
0.9914351597672121


In [29]:
len(full_df)

181321